In [1]:
import numpy as n
from scipy.optimize import root
import Properties as prop
import pandas as pd
import time

calctolerance=10**-3
calcmethod = "hybr"
KPD=0.99

#Таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#Таблица частичной нагрузки (входные данные)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

#####################################################################################################################
class heatex:
    def __init__(self,stream11,stream12,stream21,stream22):
        self.Gas_composition0={'N2':100*gas_streams0.at[stream11,'N2'],'O2':100*gas_streams0.at[stream11,'O2'],'CO2':100*gas_streams0.at[stream11,'CO2'],'H2O':100*gas_streams0.at[stream11,'H2O'],'Ar':100*gas_streams0.at[stream11,'Ar']}
        self.Gas_composition={'N2':100*gas_streams.at[stream11,'N2'],'O2':100*gas_streams.at[stream11,'O2'],'CO2':100*gas_streams.at[stream11,'CO2'],'H2O':100*gas_streams.at[stream11,'H2O'],'Ar':100*gas_streams.at[stream11,'Ar']}
        self.H011 = gas_streams0.at[stream11,'H']
        self.H012 = gas_streams0.at[stream12,'H']
        self.H021 = water_streams0.at[stream21,'H']
        self.H022 = water_streams0.at[stream22,'H']
        self.G01  = gas_streams0.at[stream11,'G']
        self.G02  = water_streams0.at[stream21,'G']
        self.P01  = gas_streams0.at[stream11,'P']
        self.P021 = water_streams0.at[stream21,'P']
        self.P022 = water_streams0.at[stream22,'P']
        self.H11  = gas_streams.at[stream11,'H']
        self.H21  = water_streams.at[stream21,'H']
        self.G1   = gas_streams.at[stream11,'G']
        self.G2   = water_streams.at[stream21,'G']
        self.P1   = gas_streams.at[stream11,'P']
        self.P21  = water_streams.at[stream21,'P']
    def calc(self):
        Q0     = self.G01*(self.H011-self.H012)*KPD
        T011   = prop.gas_T_HP(self.H011,self.P01,self.Gas_composition0)
        T012   = prop.gas_T_HP(self.H012,self.P01,self.Gas_composition0)
        T021   = prop.water_T_HP(self.H021,self.P021)
        T022   = prop.water_T_HP(self.H022,self.P022)
        dTmin0 = min(T011-T022,T012-T021)
        dTmax0 = max(T011-T022,T012-T021)
        LMTD0  = (dTmax0 - dTmin0) / (n.log(dTmax0/dTmin0))
        T01av  = (T011+T012)/2
        T02av  = (T021+T022)/2
        P02av  = (self.P021+self.P022)/2
        lambda01av = prop.gas_L_PT(self.P01,T01av,self.Gas_composition0)
        Pr01av = prop.gas_Prandtl_PT(self.P01,T01av,self.Gas_composition0)
        nu01av = prop.gas_KINVIS_PT(self.P01,T01av,self.Gas_composition0)
        ro01av = prop.gas_D_PT(self.P01,T01av,self.Gas_composition0)
        ro02av = prop.water_D_PT(P02av,T02av)
        ro021  = prop.water_DSS_P(self.P021)
        ro21   = prop.water_DSS_P(self.P21)
        ddp    = (ro21/ro021)*((self.G02/self.G2)**2)
        P22    = self.P21 - ((self.P021-self.P022)/ddp)
        P12 = self.P1
        def T12sved(T12):
            T11 = prop.gas_T_HP(self.H11,self.P1,self.Gas_composition)
            T21 = prop.water_T_HP(self.H21,self.P21)
            H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
            Q = self.G1*(self.H11-H12)*KPD
            H22 = self.H21 + (Q/self.G2)
            T22 = prop.water_T_HP(H22,P22)
            dTmin= min(T11-T22,T12-T21)
            dTmax= max(T11-T22,T12-T21)
            if dTmin<0 or dTmax<0 or dTmin==dTmax:   ##от зацикливания
                LMTD = (dTmax+dTmin)/2
            else:
                LMTD = (dTmax - dTmin) / (n.log(dTmax/dTmin))            
            dt = LMTD0/LMTD
            T1av = (T11+T12)/2
            lambda1av = prop.gas_L_PT(self.P1,T1av,self.Gas_composition)
            Pr1av = prop.gas_Prandtl_PT(self.P1,T1av,self.Gas_composition)
            nu1av = prop.gas_KINVIS_PT(self.P1,T1av,self.Gas_composition)
            ro1av = prop.gas_D_PT(self.P1,T1av,self.Gas_composition)
            kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
            Qq = Q0 / (kk*dt)
            return ((Q-Qq)/Q)
        sol = root(T12sved, T012, method=calcmethod, tol=calctolerance)
        T12=float(sol.x)
        H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
        Q = self.G1*(self.H11-H12)*KPD
        H22 = self.H21 + (Q/self.G2)
        T22 = prop.water_T_HP(H22,P22)
        return [T12,P12,H12,self.G1,T22,P22,H22,self.G2,Q]
class vapor:
    def __init__(self, stream11, stream12,stream21,stream22):
        self.Gas_composition0={'N2':100*gas_streams0.at[stream11,'N2'],'O2':100*gas_streams0.at[stream11,'O2'],'CO2':100*gas_streams0.at[stream11,'CO2'],'H2O':100*gas_streams0.at[stream11,'H2O'],'Ar':100*gas_streams0.at[stream11,'Ar']}
        self.Gas_composition={'N2':100*gas_streams.at[stream11,'N2'],'O2':100*gas_streams.at[stream11,'O2'],'CO2':100*gas_streams.at[stream11,'CO2'],'H2O':100*gas_streams.at[stream11,'H2O'],'Ar':100*gas_streams.at[stream11,'Ar']}
        self.H011 = gas_streams0.at[stream11,'H']
        self.H012 = gas_streams0.at[stream12,'H']
        self.H021 = water_streams0.at[stream21,'H']
        self.H022 = water_streams0.at[stream22,'H']
        self.G01  = gas_streams0.at[stream11,'G']
        self.G02  = water_streams0.at[stream21,'G']
        self.P01  = gas_streams0.at[stream11,'P']
        self.P02  = water_streams0.at[stream21,'P']
        self.H11  = gas_streams.at[stream11,'H']
        self.H21  = water_streams.at[stream21,'H']
        self.G1   = gas_streams.at[stream11,'G']
        self.P1   = gas_streams.at[stream11,'P']
        self.P2   = water_streams.at[stream21,'P']
    def calc(self):
        Q0     = self.G01*(self.H011-self.H012)*KPD
        T011   = prop.gas_T_HP(self.H011,self.P01,self.Gas_composition0)
        T012   = prop.gas_T_HP(self.H012,self.P01,self.Gas_composition0)
        T021   = prop.water_T_HP(self.H021,self.P02)
        T022   = prop.water_T_HP(self.H022,self.P02)
        T0np   = prop.water_TSS_P(self.P2)
        dTmin0 = min(T011-T0np,T012-T0np)
        dTmax0 = max(T011-T0np,T012-T0np)
        LMTD0  = (dTmax0 - dTmin0) / (n.log(dTmax0/dTmin0))
        T01av  = (T011+T012)/2
        lambda01av = prop.gas_L_PT(self.P01,T01av,self.Gas_composition0)
        Pr01av = prop.gas_Prandtl_PT(self.P01,T01av,self.Gas_composition0)
        nu01av = prop.gas_KINVIS_PT(self.P01,T01av,self.Gas_composition0)
        ro01av = prop.gas_D_PT(self.P01,T01av,self.Gas_composition0)
        def T12sved(T12):
            T11 = prop.gas_T_HP(self.H11,self.P1,self.Gas_composition)
            T21 = prop.water_T_HP(self.H21,self.P2)
            H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
            Q = self.G1*(self.H11-H12)*KPD
            H22 = prop.water_HSS_P(self.P2)
            T22 = prop.water_T_HP(H22,self.P2)
            G2 = Q/(H22-self.H21)
            dTmin= min(T11-T22,T12-T21)
            dTmax= max(T11-T22,T12-T21)
            if dTmin<0 or dTmax<0 or dTmin==dTmax:   ##от зацикливания
                LMTD = (dTmax+dTmin)/2
            else:
                LMTD = (dTmax - dTmin) / (n.log(dTmax/dTmin))            
            dt = LMTD0/LMTD
            T1av = (T11+T12)/2
            lambda1av = prop.gas_L_PT(self.P1,T1av,self.Gas_composition)
            Pr1av = prop.gas_Prandtl_PT(self.P1,T1av,self.Gas_composition)
            nu1av = prop.gas_KINVIS_PT(self.P1,T1av,self.Gas_composition)
            ro1av = prop.gas_D_PT(self.P1,T1av,self.Gas_composition)
            kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
            Qq = Q0 / (kk*dt)
            return ((Q-Qq)/Q)
        sol = root(T12sved, T012, method=calcmethod, tol=calctolerance)
        T12=float(sol.x)
        H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
        Q = self.G1*(self.H11-H12)*KPD
        H22 = prop.water_HSS_P(self.P2)
        T22 = prop.water_T_HP(H22,self.P2)
        G2 = Q/(H22-self.H21)
        return [T12,self.P1,H12,self.G1,T22,self.P2,H22,G2,Q]
#####################################################################################################################


#Связвка высокого давления
start_time = time.time()
it=50
for j in range(it):
    for i in range(it):
        PEVD = heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS').calc()
        gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
        water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
        IVD  = vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD').calc()
        gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
        water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
        water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
        Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
        if i>0:
            print('dQ/Q ПЕВД+ИВД',((Qgas1-Qwat1)/Qgas1))
            if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
                break;
    EVD = heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD').calc()
    gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
    water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
    
    
    Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
    Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
    if i>0:
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;

print("--- %s сек. ---" % round((time.time() - start_time)))

dQ ПЕВД+ИВД -0.38770965570277827
dQ ПЕВД+ИВД 0.04503002270354923
dQ ПЕВД+ИВД -0.017120934052255444
dQ ПЕВД+ИВД 0.006176121746631966
dQ ПЕВД+ИВД -0.002271630981798888
dQ ПЕВД+ИВД 0.0008296281809962577
dQ ПЕВД+ИВД+ЭВД 0.003204897622316441
dQ ПЕВД+ИВД -0.000560841741072584
dQ ПЕВД+ИВД+ЭВД 0.011501118758445016
dQ ПЕВД+ИВД 0.00617864768547488
dQ ПЕВД+ИВД -0.0023038653171567787
dQ ПЕВД+ИВД 0.0008529792038961765
dQ ПЕВД+ИВД+ЭВД -0.00321341889015417
dQ ПЕВД+ИВД -0.0017823686627431174
dQ ПЕВД+ИВД 0.000658568423211084
dQ ПЕВД+ИВД+ЭВД 0.0014457673427627847
dQ ПЕВД+ИВД 0.0007632419038480395
dQ ПЕВД+ИВД+ЭВД -0.00019207488805766165
--- 71 сек. ---


In [5]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,522.800000,0.1,936.681566,258.1,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,453.684298,0.1,858.022257,258.1,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,303.447446,0.1,691.393392,258.1,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,232.549267,0.1,614.739137,258.1,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,234.600000,0.1,616.940134,258.1,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,175.900000,0.1,554.284799,258.1,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,108.900000,0.1,483.533964,258.1,0.7803,0.1237,0.0301,0.0594,0.0065


In [6]:
water_streams

,T,P,H,G
PEVD-DROS,507.166885,8.258679,3414.185523,30.398495
IVD-PEVD,297.938037,8.341200,2753.507839,30.398495
EVD-IVD,297.938037,8.341200,1351.983871,30.398495
PEN-EVD,166.400000,8.341200,707.655323,30.398495
BND-PEN,NaN,0.692500,NaN,61.510000
PPND-TURB,212.300000,0.687000,2873.213987,15.510000
IND-PPND,164.512241,0.692500,2762.297432,15.510000
GPK-IND,164.000000,0.692500,692.882238,15.510000
X-GPK,60.000000,0.692500,251.745121,77.020000


In [7]:
print(prop.water_Q_HP(water_streams.at['EVD-IVD','H'],water_streams.at['EVD-IVD','P']))

0.013013706861358627
